In [3]:
import numpy as np
import cv2
import tensorflow as tf

2024-06-05 15:04:07.094120: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
mask_model = tf.keras.models.load_model('Face Mask Detection Model.h5')

In [6]:
face_detection_file = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [1]:
cap = cv2.VideoCapture(0)

while True:
    _,frame = cap.read()
    grey_face = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection_file.detectMultiScale(grey_face,1.6)
    for x,y,w,h in faces:
        face = frame[y:y+h,x:x+w]
        cv2.imwrite('face.jpg',face)
        face = tf.keras.preprocessing.image.load_img('face.jpg',target_size=(64,64,3))
        face_arr = tf.keras.preprocessing.image.img_to_array(face)
        face_arr = np.expand_dims(face_arr,axis=0)
        
        prediction = mask_model.predict(face_arr) 
        
        if prediction>0.5:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(frame,"No mask", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(frame,"Mask", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow("window",frame)
    if cv2.waitKey(1)==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()